In [1]:
import torch
from torch_geometric.datasets import QM9
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F



/home/dipplestix/Projects/rob599_final/.conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Original dataset
dataset = QM9(root='data/QM9')

total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size
print(f"Original dataset - Train size: {train_size}, Val size: {val_size}, Test size: {test_size}")

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])


batch_size = 128

# DataLoaders for original dataset
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


for batch in train_loader:
    break


Original dataset - Train size: 104664, Val size: 13083, Test size: 13084


In [3]:
def create_adjacency_matrix(batch, molecule_index):
    # Create adjacency matrix filled with zeros
    num_nodes = batch[molecule_index].num_nodes
    adj_matrix = torch.zeros((num_nodes, num_nodes))

    # Fill adjacency matrix with bond numbers from edge attributes
    edge_index = batch[molecule_index].edge_index
    edge_attr = batch[molecule_index].edge_attr

    for i in range(edge_index.shape[1]):
        start, end = edge_index[0,i], edge_index[1,i]
        # Get bond type (assuming first dimension of edge_attr encodes bond type)
        bond_type = torch.argmax(edge_attr[i]) + 1
        # Make adjacency matrix symmetric
        adj_matrix[start,end] = bond_type
        adj_matrix[end,start] = bond_type

    return adj_matrix

In [4]:
from egnn.egnn import EGNN

class HeatCapacityNet(nn.Module):
    def __init__(self, embedding_dim, num_layers, node_dim=None):
        super().__init__()
        self.atom_embedding = nn.Embedding(num_embeddings=10, embedding_dim=embedding_dim)
        if node_dim is None:   
            node_dim = embedding_dim
        self.egnn = EGNN(num_layers=num_layers, node_dim=node_dim, message_dim=embedding_dim, hidden_dim=embedding_dim)
        self.sol = nn.Sequential(
            nn.LazyLinear(128),
            nn.ReLU(),
            nn.LazyLinear(1)
        )

    def forward(self, z, r, adj, x=None):
        if x is None:
            embs = self.atom_embedding(z)
        else:
            embs = x
        h, r = self.egnn(embs, r, adj)
        hc = self.sol(h.mean(dim=0))
        return hc


In [5]:
net = HeatCapacityNet(embedding_dim=256, num_layers=10)
net(batch[0].z, batch[0].pos, create_adjacency_matrix(batch, 0)).shape

torch.Size([1])

In [6]:
from tqdm import tqdm
import torch
import torch.nn.functional as F

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print(f"GPU available: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU available, using CPU")

net = HeatCapacityNet(embedding_dim=128, num_layers=3).to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4)

best_val_mae = float('inf')
best_model_state = None

for e in range(100):
    for batch in tqdm(train_loader, desc=f"Epoch {e+1}/20", leave=False):
        loss = 0
        for i in range(len(batch)):
            initial_r = batch[i].pos.to(device)
            adj_matrix = create_adjacency_matrix(batch, i).to(device)
            pred_sol = net(batch[i].z.to(device), initial_r, adj_matrix)
            target_sol = batch[i].y[0][11].to(device)
            loss += F.mse_loss(pred_sol, target_sol.unsqueeze(0))
        loss = loss/len(batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Compute validation MAE
    val_mae = 0
    total_items = 0
    with torch.no_grad():
        for val_batch in tqdm(val_loader, desc="Validation", leave=False):
            for i, x in enumerate(val_batch):
                initial_r = val_batch[i].pos.to(device)
                adj_matrix = create_adjacency_matrix(val_batch, i).to(device)
                pred_sol = net(val_batch[i].z.to(device), initial_r, adj_matrix)
                target_sol = val_batch[i].y[0][11].to(device)
                val_mae += F.l1_loss(pred_sol, target_sol.unsqueeze(0), reduction='sum')
            total_items += len(val_batch)
    val_mae = val_mae / total_items
    
    print(f"Epoch {e+1}/20, Validation MAE: {val_mae.item():.4f}")
    
    # Save the best model
    if val_mae < best_val_mae:
        best_val_mae = val_mae
        best_model_state = net.state_dict()

# Save the best model
torch.save(best_model_state, 'best_model.pth')
print(f"Best model saved with validation MAE: {best_val_mae:.4f}")


GPU available: NVIDIA GeForce RTX 4090


Epoch 1/20, Validation MAE: 0.1624


Epoch 2/20, Validation MAE: 0.1453


Epoch 3/20, Validation MAE: 0.1166


Epoch 4/20, Validation MAE: 0.1042


...

Epoch 96/20, Validation MAE: 0.0688


Epoch 97/20, Validation MAE: 0.0568


Epoch 98/20, Validation MAE: 0.0474


Epoch 99/20, Validation MAE: 0.0454


Epoch 100/20, Validation MAE: 0.0437
Best model saved with validation MAE: 0.0366
